# This Notebook is the project deliverable for my completion of the <span style="text-decoration: underline"> Coursera Applied Data Science </span> course. 

Included in this Notebook are:
1. Capstone Project - The Battle of Neighborhoods - Week 1
2. Capstone Project - The Battle of Neighborhoods - Week 2

## This section is <span style="text-decoration: underline">The Battle of Neighborhoods - Week 1 </span>
    
For this step of the assignment I'm using data from https://data.census.gov , https://nces.ed.gov/ccd , and https://foursquare.com .


## The problem presented is: How to select an elementary school in Park City, Utah using data available for the schools and neighborhoods. ## 
#### The challenge in selecting a neighborhood based on school performance and offerings is common to parents moving to new homes in unfamiliar areas. This project provides a method for scoring schools and neighborhoods using publicly available data from government sources. ####

## The data used in the project is primarily from US government sites including NCES.ED and the US Census. ## 
The data from https://nces.ed.gov includes details such as the school level (Elementary, Middle, High), type (regular or special ed), indicator for charter schools, the TitleI status, dual language offerings, etc.   
The data from https://data.census.gov is used for information on the affluence of the neighborhoods using income statistics.   
Additional data may be used to indicate crime statistics but I have not yet located such data for Park City.   
Finally, https:foursquare.com is used to identify additional offerings or detriments proximal to the various schools and neighborhoods.


### End of Capstone Project - The Battle of Neighborhoods - Week 1

## This section is <span style="text-decoration: underline">The Battle of Neighborhoods - Week 2 </span>

In [1]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
from folium.features import Choropleth
from geopy.geocoders import Nominatim
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [46]:
pd.options.display.max_columns = None     # make sure all dataframe columns can be inspected when peeking
pd.options.display.max_rows = None        # make sure all dataframe rows can be inspected when peeking

In [5]:
# start with some school related data in ccd_sch_029_1718_w_1a_083118.csv and ccd_sch_129_1718_w_1a_083118.csv
raw=pd.read_csv(r'C:\Education\Files\ccd_sch_029_1718_w_1a_083118.csv')
raw2=pd.read_csv(r'C:\Education\Files\ccd_sch_129_1718_w_1a_083118.csv')

In [80]:
raw2.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT
0,2017-2018,1,ALABAMA,AL,Albertville Middle School,1,NaN,AL-101,100005,AL-101-0010,10000500870,100870,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual
1,2017-2018,1,ALABAMA,AL,Albertville High School,1,NaN,AL-101,100005,AL-101-0020,10000500871,100871,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual
2,2017-2018,1,ALABAMA,AL,Evans Elementary School,1,NaN,AL-101,100005,AL-101-0110,10000500879,100879,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual
3,2017-2018,1,ALABAMA,AL,Albertville Elementary School,1,NaN,AL-101,100005,AL-101-0200,10000500889,100889,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual
4,2017-2018,1,ALABAMA,AL,Big Spring Lake Kindergarten School,1,NaN,AL-101,100005,AL-101-0035,10000501616,101616,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPCEO,Yes under Community Eligibility Option (CEO),NOTVIRTUAL,Not Virtual


In [71]:
df1=raw.loc[raw['MCITY']=='PARK CITY']                        # get all the park city school info
df2=raw2.loc[raw2['STATENAME']=='UTAH']                       # get additional school info

In [72]:
df1.reset_index(drop=True,inplace=True)
df2.reset_index(drop=True,inplace=True)

In [73]:
df1.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2017-2018,49,UTAH,UT,Weilenmann School of Discovery,Weilenmann School of Discovery,1,NaN,UT-6G,4900145,UT-6G-6G100,490014501378,4901378,4199 W KILBY ROAD,NaN,NaN,PARK CITY,UT,84098,NaN,4199 W KILBY ROAD,NaN,NaN,PARK CITY,UT,84098,NaN,(435)575-5411,http://wsdpc.org/,1,Open,1,Open,05/15/2018,Regular School,1,No,No,Yes,99,USBE,NaN,NaN,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,KG,08,Elementary,As reported
1,2017-2018,49,UTAH,UT,Winter Sports School,Winter Sports School,1,NaN,UT-8I,4900168,UT-8I-8I700,490016801444,4901444,4251 Shadow Mountain Drive,NaN,NaN,PARK CITY,UT,84098,NaN,4251 Shadow Mountain Drive,NaN,NaN,PARK CITY,UT,84098,NaN,(435)649-8760,http://www.wintersportsschool.org,1,Open,1,Open,05/15/2018,Regular School,1,No,No,Yes,99,USBE,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,09,12,High,As reported
2,2017-2018,49,UTAH,UT,Jeremy Ranch School,Park City District,1,NaN,UT-22,4900750,UT-22-22112,490075000197,4900197,3050 RASMUSSEN RD,NaN,NaN,PARK CITY,UT,84098,NaN,3050 RASMUSSEN RD,NaN,NaN,PARK CITY,UT,84098,NaN,(435)645-5670,http://jres.pcschools.us/,1,Open,1,Open,05/15/2018,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,KG,05,Elementary,As reported
3,2017-2018,49,UTAH,UT,Park City High,Park City District,1,NaN,UT-22,4900750,UT-22-22704,490075000447,4900447,1750 KEARNS BLVD,NaN,NaN,PARK CITY,UT,84060,NaN,1750 KEARNS BLVD,NaN,NaN,PARK CITY,UT,84060,NaN,(435)645-5650,http://pchs.parkcity.k12.ut.us/,1,Open,1,Open,05/15/2018,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,No,No,No,10,12,High,As reported
4,2017-2018,49,UTAH,UT,Ecker Hill Middle,Park City District,1,NaN,UT-22,4900750,UT-22-22308,490075000485,4900485,2465 W KILBY RD,NaN,NaN,PARK CITY,UT,84098,NaN,2465 W KILBY RD,NaN,NaN,PARK CITY,UT,84098,NaN,(435)645-5610,http://ehms.pcschools.us/,1,Open,1,Open,05/15/2018,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,06,07,Middle,As reported


In [74]:
df2.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT
0,2017-2018,49,UTAH,UT,American Preparatory Academy - Draper #1,1,NaN,UT-74,4900005,UT-74-74100,490000500801,4900801,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,Not Virtual
1,2017-2018,49,UTAH,UT,American Preparatory Academy - The School for ...,1,NaN,UT-74,4900005,UT-74-74120,490000501372,4901372,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,Not Virtual
2,2017-2018,49,UTAH,UT,American Preparatory Academy - Accelerated School,1,NaN,UT-74,4900005,UT-74-74110,490000501404,4901404,No,SWELIGSWPROG,Title I schoolwide school,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,Not Virtual
3,2017-2018,49,UTAH,UT,American Preparatory Academy - Draper #2,1,NaN,UT-74,4900005,UT-74-74130,490000501446,4901446,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual
4,2017-2018,49,UTAH,UT,American Preparatory Academy - Draper #3,1,NaN,UT-74,4900005,UT-74-74700,490000501471,4901471,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,Not Virtual


In [82]:
df1=pd.merge(df1, df2, on="SCHID")     # combine school stats into one dataframe based on school id
df1.reset_index(drop=True,inplace=True)
df1.head(10)

,SCHOOL_YEAR_x,FIPST_x,STATENAME_x,ST_x,SCH_NAME_x,LEA_NAME,STATE_AGENCY_NO_x,UNION_x,ST_LEAID_x,LEAID_x,ST_SCHID_x,NCESSCH_x,SCHID,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED,SCHOOL_YEAR_y,FIPST_y,STATENAME_y,ST_y,SCH_NAME_y,STATE_AGENCY_NO_y,UNION_y,ST_LEAID_y,LEAID_y,ST_SCHID_y,NCESSCH_y,SHARED_TIME_x,TITLEI_STATUS_x,TITLEI_STATUS_TEXT_x,MAGNET_TEXT_x,NSLP_STATUS_x,NSLP_STATUS_TEXT_x,VIRTUAL_x,VIRTUAL_TEXT_x,SCHOOL_YEAR_x,FIPST_x,STATENAME_x,ST_x,SCH_NAME_x,STATE_AGENCY_NO_x,UNION_x,ST_LEAID_x,LEAID_x,ST_SCHID_x,NCESSCH_x,SHARED_TIME_y,TITLEI_STATUS_y,TITLEI_STATUS_TEXT_y,MAGNET_TEXT_y,NSLP_STATUS_y,NSLP_STATUS_TEXT_y,VIRTUAL_y,VIRTUAL_TEXT_y,SCHOOL_YEAR_y,FIPST_y,STATENAME_y,ST_y,SCH_NAME_y,STATE_AGENCY_NO_y,UNION_y,ST_LEAID_y,LEAID_y,ST_SCHID_y,NCESSCH_y,SHARED_TIME_x,TITLEI_STATUS_x,TITLEI_STATUS_TEXT_x,MAGNET_TEXT_x,NSLP_STATUS_x,NSLP_STATUS_TEXT_x,VIRTUAL_x,VIRTUAL_TEXT_x,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SHARED_TIME_y,TITLEI_STATUS_y,TITLEI_STATUS_TEXT_y,MAGNET_TEXT_y,NSLP_STATUS_y,NSLP_STATUS_TEXT_y,VIRTUAL_y,VIRTUAL_TEXT_y
0,2017-2018,49,UTAH,UT,Weilenmann School of Discovery,Weilenmann School of Discovery,1,NaN,UT-6G,4900145,UT-6G-6G100,490014501378,4901378,4199 W KILBY ROAD,NaN,NaN,PARK CITY,UT,84098,NaN,4199 W KILBY ROAD,NaN,NaN,PARK CITY,UT,84098,NaN,(435)575-5411,http://wsdpc.org/,1,Open,1,Open,05/15/2018,Regular School,1,No,No,Yes,99,USBE,NaN,NaN,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,KG,08,Elementary,As reported,2017-2018,49,UTAH,UT,Weilenmann School of Discovery,1,NaN,UT-6G,4900145,UT-6G-6G100,490014501378,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual,2017-2018,49,UTAH,UT,Weilenmann School of Discovery,1,NaN,UT-6G,4900145,UT-6G-6G100,490014501378,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual,2017-2018,49,UTAH,UT,Weilenmann School of Discovery,1,NaN,UT-6G,4900145,UT-6G-6G100,490014501378,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual,2017-2018,49,UTAH,UT,Weilenmann School of Discovery,1,NaN,UT-6G,4900145,UT-6G-6G100,490014501378,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual
1,2017-2018,49,UTAH,UT,Winter Sports School,Winter Sports School,1,NaN,UT-8I,4900168,UT-8I-8I700,490016801444,4901444,4251 Shadow Mountain Drive,NaN,NaN,PARK CITY,UT,84098,NaN,4251 Shadow Mountain Drive,NaN,NaN,PARK CITY,UT,84098,NaN,(435)649-8760,http://www.wintersportsschool.org,1,Open,1,Open,05/15/2018,Regular School,1,No,No,Yes,99,USBE,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,09,12,High,As reported,2017-2018,49,UTAH,UT,Winter Sports School,1,NaN,UT-8I,4900168,UT-8I-8I700,490016801444,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual,2017-2018,49,UTAH,UT,Winter Sports School,1,NaN,UT-8I,4900168,UT-8I-8I700,490016801444,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual,2017-2018,49,UTAH,UT,Winter Sports School,1,NaN,UT-8I,4900168,UT-8I-8I700,490016801444,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual,2017-2018,49,UTAH,UT,Winter Sports School,1,NaN,UT-8I,4900168,UT-8I-8I700,490016801444,No,NOTTITLE1ELIG,Not a Title I school,No,NSLPNO,No,NOTVIRTUAL,Not Virtual
2,2017-2018,49,UTAH,UT,Jeremy Ranch School,Park City District,1,NaN,UT-22,4900750,UT-22-22112,490075000197,4900197,3050 RASMUSSEN RD,NaN,NaN,PARK CITY,UT,84098,NaN,3050 RASMUSSEN RD,NaN,NaN,PARK C

In [49]:
# get the park city coordinates
address = 'Park City, UT'
geolocator = Nominatim(user_agent="parkcity_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Park City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Park City are 40.6460635, -111.4979741.


In [55]:
# create map of the Park City area 
map_park_city = folium.Map(location=[latitude, longitude], zoom_start=14)
  
map_park_city

In [ ]:
# add neighborhood and school markers to map
for lat, lng, school, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['School'], df1['Neighborhood']):
    label = '{}, {}'.format(school,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_park_city)  
    
map_park_city

### End of Capstone Project - The Battle of Neighborhoods - Week 2
### End of Notebook